In [25]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

def get_imlist(path):   #此函数读取特定文件夹下的bmp格式图像
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg')]

def get_img(path):
    c=get_imlist(path)
    d=len(c)    #这可以以输出图像个数

    data=np.empty((d,3*32*32)) #建立d*（32*32）的矩阵
    while d>0:
        img=Image.open(c[d-1])  #打开图像
        img_ndarray=np.asarray(img)
        #img_ndarray=np.asarray(img,dtype='float64')/256  #将图像转化为数组并将像素转化到0-1之间
        data[d-1]=np.ndarray.flatten(img_ndarray)    #将图像的矩阵形式转化为一维数组保存到data中
        d=d-1

    A=np.array(data).reshape(len(c),3,32,32)   #将一维数组转化为矩28*28矩阵
    return A
 
#提取数据
x_train_1=get_img(r".\dataset\train\bird")    #r""是防止字符串转译
x_train_2=get_img(r".\dataset\train\car")
x_train=np.concatenate((x_train_1,x_train_2))
y_train=[]
for i in range(0,750):
    y_train.append(1)
for i in range(750,1500):
    y_train.append(2)
#print(y_train)
y_train=np.array(y_train)
x_test_1=get_img(r".\dataset\test\bird")
x_test_2=get_img(r".\dataset\test\car")
x_test=np.concatenate((x_test_1,x_test_2))
y_test=[]
for i in range(0,100):
    y_test.append(1)
for i in range(100,200):
    y_test.append(2)
y_test=np.array(y_test)

class  K_NearestNeighbor():
    def _init_(self):
        pass
    #knn训练过程：复杂度O(1) 只需要输入数据并记录即可"""
    def train(self,X,y):
        #输入X为 N*D维 N为样本集大小 每个样本有D维"""
        self.Xtrain = X 
        #输入y为 1*N维 N为样本集大小"""
        self.Ytrain = y
    def compute_dist_one_loop(self,Xtest):
        num_test = Xtest.shape[0]
        num_train = self.Xtrain.shape[0]
        dists = np.zeros((num_test,num_train))        
        for i in range(num_test):
                dists[i,:] = np.sqrt(np.sum(np.square(self.Xtrain-Xtest[i,:]),axis=1))
        return dists 
    def predict_labels(self,Xtest,k):
        num_test = Xtest.shape[0]
        dists = self.compute_dist_one_loop(Xtest)
        y_pred = np.zeros(num_test,dtype = self.Ytrain.dtype)
        for i in range(num_test):
            # A list of length k storing the labels of the k nearest neighbors to
            # the ith test point.
            closest_y = []
                    # 排序
            idx = np.argsort(dists[i, :], axis=0)
                    # 取前 k 个最小的数据
            closest_y = self.Ytrain[idx[:k]]
                    # 找到个数出现最多的标签
            y_pred[i] = np.argmax(np.bincount(closest_y))
        return y_pred
#截断数据训练
num_training = 1500
mask = list(range(num_training))
x_train = x_train[mask]
y_train = y_train[mask]

num_test = 200
mask = list(range(num_test))
x_test = x_test[mask]
y_test = y_test[mask]
 
#为了欧氏距离的计算，我们把得到的图像数据拉长成行向量32*32*3 -> 1*3072，代码如下：

x_train = np.reshape(x_train,(x_train.shape[0],-1)) 
x_test  = np.reshape(x_test,(x_test.shape[0],-1)) 
 
#print(x_train.shape,x_test.shape)
 
#测试集预测
#计算当前测试图像和每个训练集中图像的欧氏距离
#KNN
classifier=K_NearestNeighbor() 
classifier.train(x_train,y_train) 
#y_test_pred=classifier.predict(x_test) 
 
k=10
print('predicting...')
y_test_pred=classifier.predict_labels(x_test,k) 
 
print('predicted:')
print(y_test_pred)
print('original:')
print(y_test)
 
#计算准确度
num_correct = np.sum(y_test_pred == y_test)
accu = float(num_correct)/num_test
print('K=%d accuracy %f in %d images of test' %(k,accu,num_test))

predicting...
predicted:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 2 1 2 1
 1 1 1 1 2 1 1 1 2 1 2 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1
 1 2 1 1 1 1 2 1 1 2 2 1 1 1 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2
 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1]
original:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
K=10 accuracy 0.605000 in 200 images of test
